# Facies Classification using ML
## Comparison of LSTM to best from 2016 competition

In this approach for solving the facies classfication problem ( https://github.com/seg/2016-ml-contest. ) we will explore the following statregies:
- Features Exploration: based on [Paolo Bestagini's work](https://github.com/seg/2016-ml-contest/blob/master/ispl/facies_classification_try02.ipynb), we will consider imputation, normalization and augmentation routines for the initial features.
- Model tuning: 

## Libraries

We will need to install the following libraries and packages.

In [76]:
from __future__ import print_function
import numpy as np
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold , StratifiedKFold
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
from sklearn import preprocessing
from sklearn.model_selection import LeavePGroupsOut
from sklearn.multiclass import OneVsOneClassifier
from sklearn.ensemble import RandomForestClassifier
from scipy.signal import medfilt

from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.utils import to_categorical
import numpy as np


from sklearn.ensemble import  RandomForestClassifier, VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import FunctionTransformer
import xgboost as xgb
from xgboost.sklearn import  XGBClassifier

## Data Preprocessing

In [120]:
#Load Data
data = pd.read_csv('../facies_vectors.csv')

# Parameters
feature_names = ['GR', 'ILD_log10', 'DeltaPHI', 'PHIND', 'PE', 'NM_M', 'RELPOS']
facies_names = ['SS', 'CSiS', 'FSiS', 'SiSh', 'MS', 'WS', 'D', 'PS', 'BS']
facies_colors = ['#F4D03F', '#F5B041','#DC7633','#6E2C00', '#1B4F72','#2E86C1', '#AED6F1', '#A569BD', '#196F3D']

# Store features and labels
X = data[feature_names].values 
y = data['Facies'].values 

# Store features and labels
X_ho = data_validation[feature_names].values 
y_ho = data_validation['Facies'].values 

# Store well labels and depths
well_ho = data_validation['Well Name'].values
depth_ho = data_validation['Depth'].values


# Store well labels and depths
well = data['Well Name'].values
depth = data['Depth'].values

# Fill 'PE' missing values with mean
imp = preprocessing.Imputer(missing_values='NaN', strategy='mean', axis=0)
imp.fit(X)
X = imp.transform(X)

In [122]:
#Load testing data
test_data = pd.read_csv('../validation_data_nofacies.csv')

# Prepare test data
well_ts = test_data['Well Name'].values
depth_ts = test_data['Depth'].values
X_ts = test_data[feature_names].values

In [121]:
X_ho

array([[ 66.276,   0.63 ,   3.3  , ...,   3.591,   1.   ,   1.   ],
       [ 77.252,   0.585,   6.5  , ...,   3.341,   1.   ,   0.978],
       [ 82.899,   0.566,   9.4  , ...,   3.064,   1.   ,   0.956],
       ..., 
       [ 90.49 ,   0.53 ,   6.36 , ...,   3.168,   1.   ,   0.583],
       [ 90.975,   0.522,   7.035, ...,   3.154,   1.   ,   0.556],
       [ 90.108,   0.513,   7.505, ...,   3.125,   1.   ,   0.528]])

In [79]:
X_ho.shape

(800, 7)

In [80]:
y_ho.shape

(800,)

In [81]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4149 entries, 0 to 4148
Data columns (total 11 columns):
Facies       4149 non-null int64
Formation    4149 non-null object
Well Name    4149 non-null object
Depth        4149 non-null float64
GR           4149 non-null float64
ILD_log10    4149 non-null float64
DeltaPHI     4149 non-null float64
PHIND        4149 non-null float64
PE           3232 non-null float64
NM_M         4149 non-null int64
RELPOS       4149 non-null float64
dtypes: float64(7), int64(2), object(2)
memory usage: 356.6+ KB


We procceed to run [Paolo Bestagini's routine](https://github.com/seg/2016-ml-contest/blob/master/ispl/facies_classification_try02.ipynb) to include a small window of values to acount for the spatial component in the log analysis, as well as the gradient information with respect to depth. This will be our prepared training dataset.

In [82]:
# Feature windows concatenation function
def augment_features_window(X, N_neig):
    
    # Parameters
    N_row = X.shape[0]
    N_feat = X.shape[1]

    # Zero padding
    X = np.vstack((np.zeros((N_neig, N_feat)), X, (np.zeros((N_neig, N_feat)))))

    # Loop over windows
    X_aug = np.zeros((N_row, N_feat*(2*N_neig+1)))
    for r in np.arange(N_row)+N_neig:
        this_row = []
        for c in np.arange(-N_neig,N_neig+1):
            this_row = np.hstack((this_row, X[r+c]))
        X_aug[r-N_neig] = this_row

    return X_aug


# Feature gradient computation function
def augment_features_gradient(X, depth):
    
    # Compute features gradient
    d_diff = np.diff(depth).reshape((-1, 1))
    d_diff[d_diff==0] = 0.001
    X_diff = np.diff(X, axis=0)
    X_grad = X_diff / d_diff
        
    # Compensate for last missing value
    X_grad = np.concatenate((X_grad, np.zeros((1, X_grad.shape[1]))))
    
    return X_grad


# Feature augmentation function
def augment_features(X, well, depth, N_neig=1):
    
    # Augment features
    X_aug = np.zeros((X.shape[0], X.shape[1]*(N_neig*2+2)))
    for w in np.unique(well):
        w_idx = np.where(well == w)[0]
        X_aug_win = augment_features_window(X[w_idx, :], N_neig)
        X_aug_grad = augment_features_gradient(X[w_idx, :], depth[w_idx])
        X_aug[w_idx, :] = np.concatenate((X_aug_win, X_aug_grad), axis=1)
    
    # Find padded rows
    padded_rows = np.unique(np.where(X_aug[:, 0:7] == np.zeros((1, 7)))[0])
    
    return X_aug, padded_rows

In [83]:
X_aug, padded_rows = augment_features(X, well, depth)

In [84]:
def preprocess():
    
    # Preprocess data to use in model
    X_train_aux = []
    X_test_aux = []
    y_train_aux = []
    y_test_aux = []
    
    # For each data split
    split = split_list[5]
        
    # Remove padded rows
    split_train_no_pad = np.setdiff1d(split['train'], padded_rows)

    # Select training and validation data from current split
    X_tr = X_aug[split_train_no_pad, :]
    X_v = X_aug[split['val'], :]
    y_tr = y[split_train_no_pad]
    y_v = y[split['val']]

    # Select well labels for validation data
    well_v = well[split['val']]

    # Feature normalization
    scaler = preprocessing.RobustScaler(quantile_range=(25.0, 75.0)).fit(X_tr)
    X_tr = scaler.transform(X_tr)
    X_v = scaler.transform(X_v)
        
    X_train_aux.append( X_tr )
    X_test_aux.append( X_v )
    y_train_aux.append( y_tr )
    y_test_aux.append (  y_v )
    
    X_train = np.concatenate( X_train_aux )
    X_test = np.concatenate ( X_test_aux )
    y_train = np.concatenate ( y_train_aux )
    y_test = np.concatenate ( y_test_aux )
    
    return X_train , X_test , y_train , y_test 

## Data Analysis

In this section we will run a Cross Validation routine 

In [98]:
# Train and test a classifier

# Pass in the classifier so we can iterate over many seed later.
def train_and_test_non_validation(X_tr, y_tr, X_v, well_v, clf):
    
    # Feature normalization
    scaler = preprocessing.RobustScaler(quantile_range=(25.0, 75.0)).fit(X_tr)
    X_tr = scaler.transform(X_tr)
    X_v = scaler.transform(X_v)
    
    clf.fit(X_tr, y_tr)
    
    # Test classifier
    y_v_hat = clf.predict(X_v)
    
#     Clean isolated facies for each well
    for w in np.unique(well_v):
        y_v_hat[well_v==w] = medfilt(y_v_hat[well_v==w], kernel_size=5)
    
    return y_v_hat

## Prediction

In [106]:
   
y_pred = []
print('.' * 100)
for seed in range(10):
    np.random.seed(seed)

    # Make training data.
    X_train, padded_rows = augment_features(X, well, depth)
    y_train = y
    
    X_test, padded_rows_ho = augment_features(X_ho, well_ho, depth_ho)
    y_test = y_ho
    
    X_test_nv = X_test
    y_test_nv = y_test
    
    X_train_nv = np.delete(X_train, padded_rows, axis=0)
    y_train_nv = np.delete(y_train, padded_rows, axis=0) 

    # Train classifier  
    clf = make_pipeline(XGBClassifier(learning_rate=0.12,
                                      max_depth=3,
                                      min_child_weight=10,
                                      n_estimators=150,
                                      seed=seed,
                                      colsample_bytree=0.9))


....................................................................................................


In [107]:
well_ho.shape

(800,)

In [108]:
X_test_nv.shape

(800, 28)

In [109]:
y_pred = train_and_test_non_validation(X_train_nv, y_train_nv, X_test_nv, well_ho, clf)

In [111]:
f1_score(y_pred, y_test_nv, average='micro')

0.018749999999999999

In [113]:
y_test_nv[0:20]

array([3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3])

In [119]:
y_pred

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2,
       2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 3, 1, 1, 1, 1, 1,
       1, 1, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1,

### attempt at using LSTM for including influence of previous features

In [19]:
# Feature windows concatenation function
def augment_features_window(X, N_neig):
    
    # Parameters
    N_row = X.shape[0]
    N_feat = X.shape[1]

    # Zero padding
    X = np.vstack((np.zeros((N_neig, N_feat)), X, (np.zeros((N_neig, N_feat)))))

    # Loop over windows
    X_aug = np.zeros((N_row, (2*N_neig+1), N_feat))
    for r in np.arange(N_row)+N_neig:
        this_row = []
        for c in np.arange(-N_neig,N_neig+1):
            this_row = np.hstack((this_row, X[r+c]))
#         print(this_row.shape)
        this_row.shape = ((2*N_neig+1), this_row.size // (2*N_neig+1))
#         print(this_row)
        X_aug[r-N_neig] = this_row

    return X_aug


# Feature augmentation function
def augment_features(X, well, depth, N_neig=6):
    
    # Augment features
    X_aug = np.zeros((X.shape[0], (N_neig*2+1), X.shape[1]))
    for w in np.unique(well):
        w_idx = np.where(well == w)[0]
        X_aug_win = augment_features_window(X[w_idx, :], N_neig)
#         X_aug_grad = augment_features_gradient(X[w_idx, :], depth[w_idx])
        X_aug[w_idx, :] = X_aug_win
    
    # Find padded rows
    padded_rows = np.unique(np.where(X_aug[:, 0:7] == np.zeros((1, 7)))[0])
    
    return X_aug, padded_rows

### Selection to apply LSTM w held out wells from competition

In [20]:
#Load Data
data = pd.read_csv('../facies_vectors.csv')
data_valid_labels = pd.read_csv('../blind_stuart_crawford_core_facies.csv')
data_valid = pd.read_csv('../validation_data_nofacies.csv')

In [21]:
data.head()

,Facies,Formation,Well Name,Depth,GR,ILD_log10,DeltaPHI,PHIND,PE,NM_M,RELPOS
0,3,A1 SH,SHRIMPLIN,2793.0,77.45,0.664,9.9,11.915,4.6,1,1.000
1,3,A1 SH,SHRIMPLIN,2793.5,78.26,0.661,14.2,12.565,4.1,1,0.979
2,3,A1 SH,SHRIMPLIN,2794.0,79.05,0.658,14.8,13.050,3.6,1,0.957
3,3,A1 SH,SHRIMPLIN,2794.5,86.10,0.655,13.9,13.115,3.5,1,0.936
4,3,A1 SH,SHRIMPLIN,2795.0,74.58,0.647,13.5,13.300,3.4,1,0.915


In [22]:
data_valid.head()

,Formation,Well Name,Depth,GR,ILD_log10,DeltaPHI,PHIND,PE,NM_M,RELPOS
0,A1 SH,STUART,2808.0,66.276,0.630,3.3,10.65,3.591,1,1.000
1,A1 SH,STUART,2808.5,77.252,0.585,6.5,11.95,3.341,1,0.978
2,A1 SH,STUART,2809.0,82.899,0.566,9.4,13.60,3.064,1,0.956
3,A1 SH,STUART,2809.5,80.671,0.593,9.5,13.25,2.977,1,0.933
4,A1 SH,STUART,2810.0,75.971,0.638,8.7,12.35,3.020,1,0.911


In [23]:
data_valid_labels.head()

,WellName,Depth.ft,LithCode,LithLabel
0,STUART,2807.5,3,NM Shly Silt
1,STUART,2808.0,3,NM Shly Silt
2,STUART,2808.5,3,NM Shly Silt
3,STUART,2809.0,3,NM Shly Silt
4,STUART,2809.5,3,NM Shly Silt


In [24]:
data_validation = pd.merge(data_valid, data_valid_labels,  how='left', left_on=['Well Name','Depth'], right_on = ['WellName','Depth.ft'])

In [27]:
data_validation.drop(['WellName','Depth.ft','LithLabel'],axis=1, inplace=True)

In [28]:
data_validation.rename(columns={"LithCode": "Facies"}, inplace=True)
data_validation.dropna(inplace=True)
data_validation = data_validation[data_validation.Facies <= 9]

In [29]:
data_validation.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 800 entries, 0 to 829
Data columns (total 11 columns):
Formation    800 non-null object
Well Name    800 non-null object
Depth        800 non-null float64
GR           800 non-null float64
ILD_log10    800 non-null float64
DeltaPHI     800 non-null float64
PHIND        800 non-null float64
PE           800 non-null float64
NM_M         800 non-null int64
RELPOS       800 non-null float64
Facies       800 non-null float64
dtypes: float64(8), int64(1), object(2)
memory usage: 75.0+ KB


In [30]:
data_validation['Facies'] = data_validation['Facies'].astype("int")

/home/conrad/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [31]:
data_validation.head()

,Formation,Well Name,Depth,GR,ILD_log10,DeltaPHI,PHIND,PE,NM_M,RELPOS,Facies
0,A1 SH,STUART,2808.0,66.276,0.630,3.3,10.65,3.591,1,1.000,3
1,A1 SH,STUART,2808.5,77.252,0.585,6.5,11.95,3.341,1,0.978,3
2,A1 SH,STUART,2809.0,82.899,0.566,9.4,13.60,3.064,1,0.956,3
3,A1 SH,STUART,2809.5,80.671,0.593,9.5,13.25,2.977,1,0.933,3
4,A1 SH,STUART,2810.0,75.971,0.638,8.7,12.35,3.020,1,0.911,3


In [32]:
data.head()

,Facies,Formation,Well Name,Depth,GR,ILD_log10,DeltaPHI,PHIND,PE,NM_M,RELPOS
0,3,A1 SH,SHRIMPLIN,2793.0,77.45,0.664,9.9,11.915,4.6,1,1.000
1,3,A1 SH,SHRIMPLIN,2793.5,78.26,0.661,14.2,12.565,4.1,1,0.979
2,3,A1 SH,SHRIMPLIN,2794.0,79.05,0.658,14.8,13.050,3.6,1,0.957
3,3,A1 SH,SHRIMPLIN,2794.5,86.10,0.655,13.9,13.115,3.5,1,0.936
4,3,A1 SH,SHRIMPLIN,2795.0,74.58,0.647,13.5,13.300,3.4,1,0.915


In [33]:
data_valid.shape

(830, 10)

In [42]:
#Load Data
# data = pd.read_csv('../facies_vectors.csv')
# data_valid_labels = pd.read_csv('../blind_stuart_crawford_core_facies.csv')
# data_valid = pd.read_csv('../validation_data_nofacies.csv')
# Parameters
feature_names = ['GR', 'ILD_log10', 'DeltaPHI', 'PHIND', 'PE', 'NM_M', 'RELPOS']
facies_names = ['SS', 'CSiS', 'FSiS', 'SiSh', 'MS', 'WS', 'D', 'PS', 'BS']
facies_colors = ['#F4D03F', '#F5B041','#DC7633','#6E2C00', '#1B4F72','#2E86C1', '#AED6F1', '#A569BD', '#196F3D']

# data.dropna(inplace=True)
# Store features and labels
X = data[feature_names].values 
y = data['Facies'].values 
y_test = data_validation['Facies'].values

# Store well labels and depths
well = data['Well Name'].values
depth = data['Depth'].values

# Fill 'PE' missing values with mean
imp = preprocessing.Imputer(missing_values='NaN', strategy='mean', axis=0)
imp.fit(X)
X = imp.transform(X)

# NEEDS TO BE CHANGED, SCALING SHOULD NOT BE DETERMINED FROM TESTING AND TRAINING SET, ONLY TRAINING
# scaler = preprocessing.RobustScaler(quantile_range=(25.0, 75.0)).fit(X)
# X = scaler.transform(X)

scaler = preprocessing.StandardScaler().fit(X)
X = scaler.transform(X)

# Store features and labels
X_ho = data_validation[feature_names].values 
y_ho = data_validation['Facies'].values 

# Store well labels and depths
well_ho = data_validation['Well Name'].values
depth_ho = data_validation['Depth'].values
# X = np.array(pd.DataFrame(X).dropna())
# Fill 'PE' missing values with mean
# imp = preprocessing.Imputer(missing_values='NaN', strategy='mean', axis=0)
# imp.fit(X)
# X_ho = imp.transform(X_ho)

# NEEDS TO BE CHANGED, SCALING SHOULD NOT BE DETERMINED FROM TESTING AND TRAINING SET, ONLY TRAINING
# scaler = preprocessing.RobustScaler(quantile_range=(25.0, 75.0)).fit(X)
# X = scaler.transform(X)

# scaler = preprocessing.StandardScaler().fit(X)
X_ho = scaler.transform(X_ho)
# X_test_nv_LSTM = scaler.transform(X)

In [43]:
pd.DataFrame(X_ho).describe()

,0,1,2,3,4,5,6
count,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000
mean,-0.246577,0.028201,-0.290419,-0.225423,-0.074546,0.328347,0.058799
std,0.918377,1.147799,0.641196,0.728082,0.818923,0.932224,0.986534
min,-1.745873,-4.462550,-2.522127,-1.590870,-2.038404,-1.037582,-1.775417
25%,-0.947584,-0.466278,-0.750569,-0.780086,-0.682219,-1.037582,-0.742653
50%,-0.230256,0.065042,-0.379667,-0.333156,-0.243435,0.963779,0.098212
75%,0.266958,0.761594,0.039819,0.206665,0.602520,0.963779,0.893719
max,5.131511,3.353877,2.293667,2.542621,3.282644,0.963779,1.668292


In [44]:
pd.DataFrame(X).describe()

,0,1,2,3,4,5,6
count,4.149000e+03,4.149000e+03,4.149000e+03,4.149000e+03,4.149000e+03,4.149000e+03,4.149000e+03
mean,-1.783341e-16,-1.458890e-16,-2.063372e-16,9.071237e-17,3.690843e-16,3.267465e-15,-1.919276e-16
std,1.000121e+00,1.000121e+00,1.000121e+00,1.000121e+00,1.000121e+00,1.000121e+00,1.000121e+00
min,-1.808152e+00,-2.713051e+00,-4.974011e+00,-1.773849e+00,-4.457407e+00,-1.037582e+00,-1.820774e+00
25%,-6.668229e-01,-6.394262e-01,-5.313459e-01,-6.591524e-01,-6.638841e-01,-1.037582e+00,-8.543031e-01
50%,1.848762e-03,-8.139271e-02,-1.943073e-02,-1.656012e-01,0.000000e+00,9.637792e-01,2.145242e-02
75%,4.786981e-01,6.428636e-01,5.872835e-01,3.994588e-01,3.477220e-01,9.637792e-01,8.623173e-01
max,9.776468e+00,4.513479e+00,2.826818e+00,9.983045e+00,5.524616e+00,9.637792e-01,1.668292e+00


In [45]:
X_train, padded_rows = augment_features(X, well, depth)
y_train = y

X_test, padded_rows = augment_features(X_ho, well_ho, depth_ho)

X_test = X_test[0:800]
y_test = y_test[0:800]
X_train = X_train[0:4120]
y_train = y_train[0:4120]
y_train = y_train - 1
y_test = y_test - 1
y_train_ct = to_categorical(np.array(y_train), num_classes=None)
y_test_ct = to_categorical(np.array(y_test), num_classes=None)

In [56]:
'''Trains an LSTM model on the IMDB sentiment classification task.
The dataset is actually too small for LSTM to be of any advantage
compared to simpler, much faster methods such as TF-IDF + LogReg.
# Notes
- RNNs are tricky. Choice of batch size is important,
choice of loss and optimizer is critical, etc.
Some configurations won't converge.
- LSTM loss decrease patterns during training can be quite different
from what you see with CNNs/MLPs/etc.
'''




data_dim = 7 # Features
timesteps = 13 # 25 is best so far observed
num_classes = 9
batch_size = 20 # 20 is best so far observed

# Expected input batch shape: (batch_size, timesteps, data_dim)
# Note that we have to provide the full batch_input_shape since the network is stateful.
# the sample of index i in batch k is the follow-up for the sample i in batch k-1.
model = Sequential()


model.add(LSTM(20, stateful=True,
               batch_input_shape=(batch_size, timesteps, data_dim)))

# model.add(LSTM(40, return_sequences=True, stateful=True))

# model.add(LSTM(40, stateful=True))
model.add(Dense(9, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='Nadam',
              metrics=['accuracy'])

model.fit(X_train, y_train_ct,
          batch_size=batch_size, epochs=30, shuffle=False,
          validation_data=(X_test, y_test_ct))

Train on 4120 samples, validate on 800 samples
Epoch 1/30
4120/4120 [==============================] - 3s 623us/step - loss: 1.7416 - acc: 0.3398 - val_loss: 1.6185 - val_acc: 0.3650
Epoch 2/30
4120/4120 [==============================] - 2s 480us/step - loss: 1.3779 - acc: 0.4602 - val_loss: 1.4742 - val_acc: 0.4363
Epoch 3/30
4120/4120 [==============================] - 2s 478us/step - loss: 1.2355 - acc: 0.5119 - val_loss: 1.4243 - val_acc: 0.4625
Epoch 4/30
4120/4120 [==============================] - 2s 476us/step - loss: 1.1552 - acc: 0.5342 - val_loss: 1.3999 - val_acc: 0.4637
Epoch 5/30
4120/4120 [==============================] - 2s 530us/step - loss: 1.0992 - acc: 0.5602 - val_loss: 1.4274 - val_acc: 0.4475
Epoch 6/30
4120/4120 [==============================] - 2s 479us/step - loss: 1.0531 - acc: 0.5748 - val_loss: 1.4362 - val_acc: 0.4488
Epoch 7/30
4120/4120 [==============================] - 2s 470us/step - loss: 1.0224 - acc: 0.5939 - val_loss: 1.3965 - val_acc: 0.4638
E